#### Q1. Getting the embeddings model
- Answer: 0.07822264

In [3]:
from sentence_transformers import SentenceTransformer

model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
user_question = "I just discovered the course. Can I still join it?"
question_v = embedding_model.encode(user_question)

In [19]:
print(question_v[0])

0.07822264


#### Q2. Creating the embeddings
- Answer: (375, 768)

In [9]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [14]:
filtered_docs = [doc for doc in documents if doc["course"] == "machine-learning-zoomcamp"]

# We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents
len(filtered_docs)

375

In [21]:
import numpy as np

embeddings = []
for index, doc in enumerate(filtered_docs):
    if index%10==0:
        print(f"Processed {index} docs...")
    
    qa_text = f'{doc["question"]} {doc["text"]}'
    doc_v = embedding_model.encode(qa_text)
    embeddings.append(doc_v)

X = np.array(embeddings)
X.shape

Processed 0 docs...
Processed 10 docs...
Processed 20 docs...
Processed 30 docs...
Processed 40 docs...
Processed 50 docs...
Processed 60 docs...
Processed 70 docs...
Processed 80 docs...
Processed 90 docs...
Processed 100 docs...
Processed 110 docs...
Processed 120 docs...
Processed 130 docs...
Processed 140 docs...
Processed 150 docs...
Processed 160 docs...
Processed 170 docs...
Processed 180 docs...
Processed 190 docs...
Processed 200 docs...
Processed 210 docs...
Processed 220 docs...
Processed 230 docs...
Processed 240 docs...
Processed 250 docs...
Processed 260 docs...
Processed 270 docs...
Processed 280 docs...
Processed 290 docs...
Processed 300 docs...
Processed 310 docs...
Processed 320 docs...
Processed 330 docs...
Processed 340 docs...
Processed 350 docs...
Processed 360 docs...
Processed 370 docs...


(375, 768)

#### Q3. Search
- Answer: 0.6506573

In [24]:
scores = X.dot(question_v)

In [26]:
max(scores)

0.6506573

In [34]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        
        # replace argsort with argpartition to make it a lot faster
        idx = np.argpartition(-scores, num_results-1)[:num_results]
        top_idx = idx[np.argsort(-scores[idx])]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_docs, embeddings=X)
search_engine.search(question_v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

#### Q4. Hit-rate for our search engine
- Answer: 0.9398907103825137

In [35]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [38]:
ground_truth[:4]
len(ground_truth)

1830

In [40]:
search_engine = VectorSearchEngine(documents=filtered_docs, embeddings=X)

validation_results = []
for index, truth in enumerate(ground_truth):
    if index%50==0:
        print(f"Processed {index} truth...")
        
    question = truth["question"]
    doc_id = truth["document"]
    
    question_v = embedding_model.encode(question)
    search_results = search_engine.search(question_v, num_results=5)
    result = any(search_result["id"] == doc_id for search_result in search_results)
    validation_results.append(result)

Processed 0 truth...
Processed 50 truth...
Processed 100 truth...
Processed 150 truth...
Processed 200 truth...
Processed 250 truth...
Processed 300 truth...
Processed 350 truth...
Processed 400 truth...
Processed 450 truth...
Processed 500 truth...
Processed 550 truth...
Processed 600 truth...
Processed 650 truth...
Processed 700 truth...
Processed 750 truth...
Processed 800 truth...
Processed 850 truth...
Processed 900 truth...
Processed 950 truth...
Processed 1000 truth...
Processed 1050 truth...
Processed 1100 truth...
Processed 1150 truth...
Processed 1200 truth...
Processed 1250 truth...
Processed 1300 truth...
Processed 1350 truth...
Processed 1400 truth...
Processed 1450 truth...
Processed 1500 truth...
Processed 1550 truth...
Processed 1600 truth...
Processed 1650 truth...
Processed 1700 truth...
Processed 1750 truth...
Processed 1800 truth...


In [42]:
sum(validation_results)/len(validation_results)

0.9398907103825137

#### Q5. Indexing with Elasticsearch
- Answer: ee58a693

In [ ]:
'''
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
'''

In [54]:
from elasticsearch import Elasticsearch, helpers
import requests

host = "http://localhost:9200"
es = Elasticsearch([host])

# 定义索引设置
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "doc_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}

# 创建索引
index_name = 'doc_vector'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_settings)

In [55]:
from elasticsearch.helpers import bulk

# filtered_docs & embeddings
actions = [
    {
        "_index": index_name,
        "_id": doc["id"],
        "_source": {
            "text": doc["text"],
            "section": doc["section"],
            "question": doc["question"],
            "course": doc["course"],
            "id": doc["id"],
            "doc_vector": emb
        }
    }
    for doc, emb in zip(filtered_docs, embeddings)
]

# 执行批量操作
bulk(es, actions)

(375, [])

In [70]:
def elastic_search_knn(field, vector, course="machine-learning-zoomcamp"):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        #print(hit)
        result_docs.append(hit['_source'])

    return result_docs

In [71]:
user_question = "I just discovered the course. Can I still join it?"
question_v = embedding_model.encode(user_question)

result_docs = elastic_search_knn('doc_vector', question_v)

In [72]:
result_docs

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

#### Q6. Hit-rate for Elasticsearch
- Answer: 0.9398907103825137

In [73]:
es_validation_results = []
for index, truth in enumerate(ground_truth):
    if index%50==0:
        print(f"Processed {index} truth...")
        
    question = truth["question"]
    doc_id = truth["document"]
    
    question_v = embedding_model.encode(question)
    search_results = elastic_search_knn('doc_vector', question_v)
    result = any(search_result["id"] == doc_id for search_result in search_results)
    es_validation_results.append(result)

Processed 0 truth...
Processed 50 truth...
Processed 100 truth...
Processed 150 truth...
Processed 200 truth...
Processed 250 truth...
Processed 300 truth...
Processed 350 truth...
Processed 400 truth...
Processed 450 truth...
Processed 500 truth...
Processed 550 truth...
Processed 600 truth...
Processed 650 truth...
Processed 700 truth...
Processed 750 truth...
Processed 800 truth...
Processed 850 truth...
Processed 900 truth...
Processed 950 truth...
Processed 1000 truth...
Processed 1050 truth...
Processed 1100 truth...
Processed 1150 truth...
Processed 1200 truth...
Processed 1250 truth...
Processed 1300 truth...
Processed 1350 truth...
Processed 1400 truth...
Processed 1450 truth...
Processed 1500 truth...
Processed 1550 truth...
Processed 1600 truth...
Processed 1650 truth...
Processed 1700 truth...
Processed 1750 truth...
Processed 1800 truth...


In [74]:
sum(es_validation_results)/len(es_validation_results)

0.9398907103825137